In [1]:
import collections
from multiprocessing import Pool
import pandas as pd
import numpy as np
import time
import os

In [2]:
data = pd.read_csv('https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv')
data.reset_index(inplace=True) #Use idx as identifier
print(data.shape[0])
data.head()

150


,index,sepal_length,sepal_width,petal_length,petal_width,species
0,0,5.1,3.5,1.4,0.2,setosa
1,1,4.9,3.0,1.4,0.2,setosa
2,2,4.7,3.2,1.3,0.2,setosa
3,3,4.6,3.1,1.5,0.2,setosa
4,4,5.0,3.6,1.4,0.2,setosa


In [7]:
### Divide into chunks of equal size 

In [8]:
## Define number of chunks
n = 10

In [23]:
### Sequential implementation
def getArea(x):
    return {'ID':x.index,
            'sepArea':x.sepal_length * x.sepal_width,
            'petArea':x.petal_length * x.petal_width}

def getAreaSlow(x,verbose=False):
    if verbose: print(f'processing: {x.index}')
    time.sleep(.01)
    result = {'ID':x.index,
            'sepArea':x.sepal_length * x.sepal_width,
            'petArea':x.petal_length * x.petal_width}
    if verbose: print(f'done with: {x.index}')
    return result

In [9]:
def getIndices(data,n):
    dataSize = range(data.shape[0])
    chunksIdx = np.array_split(dataSize, n)
    return chunksIdx

def getRanges(array):
    return array[0],array[-1]

def subsetChunks(iterable,idxArray,idx):
    a,b = getRanges(idxArray[idx])
    return iterable[a:b]

In [11]:
arrayIdxChunks = getIndices(data,n)

In [12]:
### Make iterable
iterData = list(data.itertuples())

In [51]:
### Create a dictionary of iterable chunks
chunkedIter = {_+1 : subsetChunks(iterData,arrayIdxChunks,_) for _ in range(len(arrayIdxChunks))}

In [52]:
### Create a dictionary of dataframe chunk
chunkedDF = {_+1 : subsetChunks(data,arrayIdxChunks,_) for _ in range(len(arrayIdxChunks))}

In [96]:
pd.DataFrame(list(map(getArea,iterData)))

,ID,sepArea,petArea
0,0,17.85,0.28
1,1,14.70,0.28
2,2,15.04,0.26
3,3,14.26,0.30
4,4,18.00,0.28
...,...,...,...
145,145,20.10,11.96
146,146,15.75,9.50
147,147,19.50,10.40
148,148,21.08,12.42


In [90]:
#### Write tables
for k,v in chunkedDF.items():
    out = f'out_chunk_{k}.txt'
    v.to_csv(out,sep="\t",columns=None,index=None)

In [136]:
!tail -n +2 out_chunk_1.txt | awk 'BEGIN {FS="\t"; print "ID","sepArea","petArea"} {print $1,$2*$3,$4*$5}'

ID sepArea petArea
/bin/sh: -n: command not found


### Xargs

In [3]:
out = f'out_iris_full.txt'
data.to_csv(out,sep=="\t",columns=None,index=None)

In [4]:
!cat out_iris_full.txt | xargs ./test_xargs.sh